# Data wrangling on [people_data.csv](https://piazza.com/redirect/s3?bucket=uploads&prefix=attach%2Fjsagy1god6f2bq%2Fjdl2dukjx9e4n9%2Fjswf28nz9jht%2Fpeople_data.csv) by `Mostafa Tarek` using `MongoDB`, `Pymongo` 

__vis the data and assessing it__

In [2]:
import pandas as pd 
df = pd.read_csv('./people_data.csv')
df

,ID,firstname,lastname,age,currentJob,prevJobs,friendsID,hobbies,email,nationality
0,0,magdi,abd elrahman,35.0,Surveying and Mapping Technicians,"Agricultural Engineers,Wind Energy Engineers","712,6592,7375","competitive programming,competitive programming",NaN,egyptian
1,1,samir,tamer,84.0,NaN,"Fuel Cell Engineers,Materials Engineers","1139,2309,2767,3343,9150,9533,9845",NaN,samirtamer1626@gmail.com,syrian
2,2,eslam,rashad,74.0,Mapping Technicians,NaN,"462,720,3195,3836,4704",football,rashadeslam6970@yahoo.com,iraqi
3,3,ali,eslam,NaN,"Health and Safety Engineers, Except Mining Saf...",Electro-Mechanical Technicians,"1659,3995,6073","football,camping",eslamali1859@gmail.com,iraqi
4,4,ehab,maher,NaN,Photonics Engineers,"Agricultural Engineers,Marine Engineers","2090,7126","cooking,swimming",NaN,syrian
5,5,yosra,ibrahim,NaN,Validation Engineers,NaN,4220,"swimming,football",NaN,lebanese
6,6,mona,Arshd,NaN,Mechatronics Engineers,"Chemical Engineers,Petroleum Engineers","498,1245,2567,6508,9255",NaN,NaN,moroccan
7,7,yosra,ibrahim,38.0,Energy Engineers,NaN,"2567,3746,4563,5207,9662,9695",NaN,yosraibrahim4119@hotmail.com,lebanese
8,8,mai,mazen,NaN,NaN,"Civil Engineers,Nanotechnology Engineering Tec...","992,4719,6870",cooking,mazenmai7201@yahoo.com,iraqi
9,9,sanaa,samy,NaN,Environmental Engineers,NaN,"5150,5842,8582,9426",football,NaN,syrian


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 10 columns):
ID             10000 non-null int64
firstname      10000 non-null object
lastname       10000 non-null object
age            7181 non-null float64
currentJob     8732 non-null object
prevJobs       6039 non-null object
friendsID      9734 non-null object
hobbies        6045 non-null object
email          5876 non-null object
nationality    10000 non-null object
dtypes: float64(1), int64(1), object(8)
memory usage: 781.3+ KB


In [4]:
from pymongo import MongoClient
client = MongoClient('mongodb://localhost:27017')

__create a `people` database and assign it to `peopleDb` var

In [5]:
peopleDb = client["people"]

In [6]:
client.list_database_names()

['admin', 'config', 'local', 'people']

__add `people_info` collection to `people` database__ and assign it to `peopleInfo`

In [7]:
peopleInfo = peopleDb["people_info"]

In [8]:
peopleDb.list_collection_names()

['people_info']

__assign the columns names to `columns` and remove the df after assessing and vis it to save memory__


In [9]:
columns = df.columns
del df 

In [10]:
import csv  
from pprint import pprint 

In [11]:
peopleInfo.delete_many({})
print("count before inserting {} \n inserting... ".format(peopleInfo.count_documents({})))
with open('./people_data.csv','r') as file :
    reader = csv.DictReader(file, fieldnames=columns)
    next(reader) # skipt the header 
    data = [] 
    for row in reader:
        data.append(dict(row)) 
peopleInfo.insert_many(data)
print("count after inserting {} ".format(peopleInfo.count_documents({})))

count before inserting 0 
 inserting... 
count after inserting 10000 


### 1- remove all fields with empty values 

In [12]:
c = 0 
for i in columns:
    c += peopleInfo.count_documents({i:{"$in":["NaN","nan"," ","",None,"Null","null"]}})
    peopleInfo.delete_many({i:{"$in":["NaN","nan"," ","",None,"Null","null"]}})
print("deleted {} documents".format(c))

deleted 8703 documents


In [13]:
#uncomment to vis the docs 
# for doc in peopleInfo.find({}):
#     pprint(doc)
#     pprint('\n')

### 2- change `prevjops` ,` hobbies` and `friendsID`  to array data type

In [14]:
for field in ["prevJobs" , "hobbies" , "friendsID"]:
    for doc in peopleInfo.find({field: {"$exists":1}}):
        newvalues = { "$set": {field: doc[field].split(',')}}
        peopleInfo.update_one({"_id":doc["_id"]}, newvalues)

In [15]:
#uncomment to vis 
# for doc in peopleInfo.find({}):
#     print(doc["prevJobs"])
#     print('\n')

### 3- Find all persons’ document that his `current job` is `his first job` and also his `age` between `20` and` 30 year`

__convet `age` to `float` so we can manipulate  with it__

In [16]:
type(next(peopleInfo.find( { "age": { "$exists": 1 } } ))["age"])

str

In [17]:
for doc in peopleInfo.find( { "age": { "$exists": 1 } } ):
    peopleInfo.update_one({"_id":doc["_id"]},{"$set":{"age":float(doc["age"])}})

In [18]:
type(next(peopleInfo.find( { "age": { "$exists": 1 } } ))["age"])

float

In [19]:
for doc in peopleInfo.find({ "$and":[{"currentJob":{"$exists":1}},{"prevJobs":{"$exists":1}},  { "age": { "$lte": 30 } }, { "age": {"$gte":20} }  ] }):
        if doc["currentJob"] in doc["prevJobs"]:
            #uncomment to vis 
            #print(doc)
            pass

## 4- for each email domain find how many persons using it?
__Email domains : yahoo , gmail , hotmail ….etc__

In [20]:
for i in peopleInfo.aggregate([
  { "$project" : { "emailDomains" : { "$split": ["$email","@"] }} },
    {"$project": { "emailDomains": {"$slice":["$emailDomains",-1]}}},
    {"$unwind": "$emailDomains"}, #so we can split it by . at the next pipline as the [] type will raise an error, in other words this step get the string from an array 
   { "$project" : { "emailDomains" : { "$split": ["$emailDomains","."] }} },
    {"$project": { "emailDomains": {"$slice":["$emailDomains",1]}, "qt":1}},
    {"$unwind": "$emailDomains"},
  { "$group" : { "_id": "$emailDomains" , "count" : { "$sum" : 1 } } },
  { "$sort" : { "total_qty" : -1 } }
]):
    print(i)

{'_id': 'hotmail', 'count': 446}
{'_id': 'yahoo', 'count': 413}
{'_id': 'gmail', 'count': 438}


## 5- for all persons with fullname : `mohamed samir` find all documents of their friends .

In [21]:
friendsDocs = [] 
for doc in peopleInfo.aggregate([
    {"$project": {"full_name": {"$concat": ["$firstname"," ","$lastname"]}, "friendsID":"$friendsID"}},
    {"$unwind":"$friendsID"},
    {"$match": {"full_name": "mohamed samir"}}
]):
    friendsDocs.append(peopleInfo.find({"ID":doc["friendsID"]}))
"number of docs is  {} docs ".format(len(friendsDocs))

'number of docs is  22 docs '

In [22]:
#uncomment to vis 
# for i in friendsDocs:
#     for j in i:
#         print(j)

- note that the above output has just `two` docs out of `22` docs as we preveously removed the docs that contain empty fields at procedure number `one` `1- remove all fields with empty values ` 

### 6.1- for all Egyptian persons , how many persons with first name `ahmed`  ? 

In [24]:
for i in peopleInfo.aggregate([
    { "$match": {"firstname":"ahmed", "nationality":"egyptian"}},
    { "$group": {"_id": "$firstname" , "count" : { "$sum" : 1 } } }
]):
    print(i["count"])

23


###  6.2- how many persons `have more than one hobby` ?

In [29]:
a =  peopleInfo.aggregate([
    {"$match": { "nationality":"egyptian"}},
    {"$project":{"hobbiesCount":{"$size":"$hobbies"}}},
    {"$match": {"hobbiesCount": {"$gt":1}} }
])
len(list(a))

116

__but there is a problem that there maybe are some redundant persons so we should group by name at the end__
### so:

In [30]:
a =  peopleInfo.aggregate([
    {"$match": { "nationality":"egyptian"}},
    {"$project":{"hobbiesCount":{"$size":"$hobbies"},  "name":{"$concat":["$firstname"," ","$lastname"]}  }},
    {"$match": {"hobbiesCount": {"$gt":1}} },
    {"$group": {"_id":"$name"}},
])
len(list(a))

109

__note there were 116 - 109 == `7` redundant persons name__

###  6.3- and  find the `top 5 jobs with respect to number of people working it` 

In [33]:
list(peopleInfo.aggregate([
    {"$match": { "nationality":"egyptian"}},
    {"$group": {"_id":"$currentJob", "count":{"$sum":1}}},
    {"$sort": {"count":-1}},
    {"$limit":5}
]))

[{'_id': 'Architects, Except Landscape and Naval', 'count': 8},
 {'_id': 'Engineers, All Other', 'count': 7},
 {'_id': 'Architectural Drafters', 'count': 7},
 {'_id': 'Human Factors Engineers and Ergonomists', 'count': 7},
 {'_id': 'Non-Destructive Testing Specialists', 'count': 7}]

# conclusion 

### Personally, I do not see any advantage for MongoDB to make it overcome SQL "in this particular task" 